In [ ]:
# default_exp core.database_rq

In [ ]:
#hide
from nbdev.showdoc import *
import json
from pathlib import Path
from pydiscogs.core import *

In [ ]:
#hide
with open(Path("../config/example_user_info.json"), "r") as user_config_file:
    config = json.load(user_config_file)
    user = UserWithUserTokenBasedAuthentication(user_token=config["user_token"],
                                                user_agent=config["user_agent"])

# Core Database Requests

> function wrapper around the discogs api for database interactions

In [ ]:
#export
import requests
from typing import Union
from pydiscogs.core import *

In [ ]:
#export
def get_release(user:UserWithUserTokenBasedAuthentication, 
                release_id:int, 
                curr_abbr=Union[str, None])->requests.models.Response:
    """Get information to a particular release from discogs database.
       A release represents a particular physical or digital object released by 
       one or more Artists."""
    url = f"{BASE_URL}/releases/{release_id}"
    headers = user.headers
    params = {"token": user.user_token} 
    if curr_abbr: 
        params["curr_addr"] = curr_abbr
    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of get_releases
get_release(user=user, release_id=6067651, curr_abbr="EUR")

<Response [200]>

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_core.constants.ipynb.
Converted 01_core.discogs_user.ipynb.
Converted 02_core.database_rq.ipynb.
Converted index.ipynb.
